In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

### Loading LLM model and Embedding model

In [3]:
from langchain_groq import ChatGroq
from sentence_transformers import SentenceTransformer

llm = ChatGroq(
    model="llama3-8b-8192"
)

embedding_model = SentenceTransformer("./sentence-transformers-local/all-miniLM-L6-v2")

C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
embedding_model.encode("Large Language Model")[:10]

array([ 0.01080922, -0.09723883,  0.01275732, -0.04699858, -0.02833704,
        0.05745238, -0.01973694,  0.03022125,  0.03715058, -0.0367018 ],
      dtype=float32)

In [5]:
llm.invoke("Hello, llm")

AIMessage(content="Hello there! It's nice to meet you. I'm a large language model (LLM), trained to understand and generate human-like text. I'm here to assist you with any questions or topics you'd like to discuss. What's on your mind?", response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 14, 'total_tokens': 67, 'completion_time': 0.041140501, 'prompt_time': 0.003943335, 'queue_time': None, 'total_time': 0.045083836}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'stop', 'logprobs': None}, id='run-c271c3ec-3bf8-40fc-8c5a-c4819ce1a3c1-0')

In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(
    model_name="./sentence-transformers-local/all-miniLM-L6-v2"
)

C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [7]:
embedding.embed_query("Large Language Model")[:10]

[0.010809220373630524,
 -0.09723883122205734,
 0.012757323682308197,
 -0.046998582780361176,
 -0.028337037190794945,
 0.05745238438248634,
 -0.019736938178539276,
 0.030221253633499146,
 0.03715058043599129,
 -0.03670179843902588]

In [8]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.chains import create_sql_query_chain
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from operator import itemgetter
from langchain_pinecone import PineconeVectorStore

### SQL Chain: (NLP to SQL query to execute)

In [9]:
db_path = "./products.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

In [10]:
import re

def extract_query(text):
    # Define the regular expression to find the SQL query part
    pattern = r'SQLQuery:\s*(.*)'
    match = re.search(pattern, text)
    
    if match:
        return match.group(1)
    else:
        return None

In [11]:
write_query = create_sql_query_chain(llm, db)
extract_sql_query = RunnableLambda(lambda x: extract_query(x))
query_getter = write_query | extract_sql_query
execute_query = QuerySQLDataBaseTool(db=db)

In [12]:
answer_prompt = PromptTemplate.from_template(
    """
    "You are customer service agent for an e-commerce store located in Nepal.The store sell electronics and gadgets and you are chatting with a customer who need help.
    Given the following user question, corresponding  SQL query, and the result of the query, respond to the user with the suitable answer to the question without adding anything up in a short and concise manner as possible.
    
    If the SQL Result is empty, dont try to make up answer, just negate the user question.
    
    Question: <question>{question}</question>
    SQL Query: <query>{query}</query>
    SQL Result: <result>{result}</result>
    Answer:
    """
)

answer_chain = answer_prompt | llm | StrOutputParser()

In [13]:
sql_chain = (
    RunnablePassthrough.assign(query=query_getter).assign(result=itemgetter("query") | execute_query)
    | answer_chain
)

In [14]:
sql_chain.invoke({
    "question": "How many products are there?"
})

'There are 9 products available in our store.'

### RAG CHAIN: (retriever)

In [15]:
os.environ["PINECONE_API_KEY"] = os.environ["PINECONE_API_KEY"]
index_name = "storeagent"

vectorstore = PineconeVectorStore(
    index_name=index_name,
    embedding=embedding,
)

In [16]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k":2
    }
)
retriever.invoke("Delivery options")

[Document(page_content='## What are the delivery options\n- The delivery  options is available all over Nepal and the delivery charge is Rs. 120 standard price and delivery is free inside Dharan area.', metadata={'source': 'info.txt'}),
 Document(page_content='## How long the delivery takes\n- The delivery takes about 2-3 days for all over Nepal and if you are within Dharan Delivery will be the same day you order product.', metadata={'source': 'info.txt'})]

In [17]:
chat_system_prompt = (
    "You are customer service agent for an e-commerce store."
    "The store sell electronics and gadgets and you are chatting with a customer who need help."
    "Use the retrieved information or chat history to answer the customer's question."
    "If the retrieved information is not useful or the chat history is not relevant, you can ignore it and just answer the question."
    "but remember you only answer about the store and questions related to it and nothing else."
    "If asked anything else just say that you are just customer service agent and you can only answer about the store."
    "\n\n"
    "{context}"
)

In [18]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", chat_system_prompt),
        ("human", "{input}"),
    ]
)

In [19]:
chat_chain = create_stuff_documents_chain(
    llm,
    chat_prompt
)

rag_chain = create_retrieval_chain(
    retriever, # retrieve documents
    chat_chain # get the retrieved documents and pass it to LLM to answer the question
)

### Classification Chain: (inquiry or info)

In [35]:
classifier_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a customer service agent who is expert in classifying the customer questions in ecommerce store."),
        ("human", "Classify the question: {question} as one of the following:\n1. 'product_inquiry': If the customer is asking about specific product price, product details or product stock only.\n2. 'general_information': If the customer is asking about the store information, store location, store policies or anything that is general information only about the store.\n3. 'others': If the question does not fall in any of the above categories and is likely out of topic for a customer support service of ecommerce store.")
    ]
)

In [36]:
classification_chain = classifier_template | llm | StrOutputParser()

### Building Final Chain: RAG + SQL (conditional)

In [20]:
general_chain = PromptTemplate.from_template(
    """
    Respond to the user that you dont answer that question as you are a customer service agent and you can only answer about the store.
    """
) | llm | StrOutputParser()

In [21]:
def route(info):
    print(info)
    if "general_information" in info["topic"].lower():
        question = info["question"]
        return RunnableLambda(lambda x: {"input": question}) | rag_chain | RunnableLambda(lambda x: x["answer"])
    elif "product_inquiry" in info["topic"].lower():
        return sql_chain
    else:
        return general_chain

In [24]:
sql_rag_chain = {
    "topic": classification_chain,
    "question": lambda x: x["question"]
} | RunnableLambda(route)

### Testing our final chain: (sql_rag_chain)

In [27]:
def chat(question):
    return sql_rag_chain.invoke({"question": question})

In [28]:
chat("Hello")

{'topic': 'I would classify the question "Hello" as \'general_information\'.', 'question': 'Hello'}


"Hello! Welcome to All Electronics Store's customer service. How can I assist you today?"

In [29]:
chat("Who are you?")

{'topic': 'I would classify the question "Who are you?" as \'general_information\'. The question is asking about the identity or background of the customer service agent, which falls under the category of general information about the store.', 'question': 'Who are you?'}


"Hello! I'm a customer service agent for an e-commerce store that sells electronics and gadgets. I'm here to help answer any questions you have about our products, services, or policies."

In [30]:
chat("What this store sell?")

{'topic': 'I would classify the question "What this store sell?" as \'general_information\'. The customer is asking about the overall nature of the store\'s products, which is a general piece of information about the store.', 'question': 'What this store sell?'}


"Hello! According to our store's information, we sell a wide range of electronics and gadgets, including smartphones, cameras, laptops, PCs, and calculators."

In [31]:
chat("What is the return policy")

{'topic': 'I would classify the question "What is the return policy?" as:\n\n2. \'general_information\'\n\nThe customer is asking about the store\'s return policy, which is a general information about the store\'s policies. This type of question is not specific to a particular product, but rather about the store\'s overall policy.', 'question': 'What is the return policy'}


'Our return policy allows customers to return products within 3 days after receiving their shipment. The product must be in the same condition as when it was purchased, meaning it should be in its original packaging and condition. If the product meets these conditions, we will be happy to assist with the return process.'

In [32]:
chat("Do you have redmi note 9 pro in your store>")

{'topic': 'I would classify the question "Do you have redmi note 9 pro in your store?" as:\n\n1. \'product_inquiry\'\n\nThe customer is asking about the availability of a specific product (Redmi Note 9 Pro) in the store, which falls under the category of product inquiry.', 'question': 'Do you have redmi note 9 pro in your store>'}


'Yes, we have Redmi Note 9 Pro in our store.'

In [33]:
chat("What is the price of calculator in your store?")

{'topic': 'I would classify the question "What is the price of calculator in your store?" as:\n\n1. \'product_inquiry\'\n\nThe customer is specifically asking about the price of a calculator in the store, which falls under the category of product inquiry.', 'question': 'What is the price of calculator in your store?'}


'Thank you for reaching out to us! According to our records, the price of the calculator in our store is NPR 1200.'

In [34]:
chat("What is the full form of AI>")

{'topic': 'I would classify the question "What is the full form of AI?" as \'general_information\'. This question falls under the category of general information, as it is not related to a specific product or store policies, but rather an inquiry about a general topic (Artificial Intelligence).', 'question': 'What is the full form of AI>'}


"Hi there! I'm happy to help you with your question. However, I'd like to clarify that AI is not related to our e-commerce store that sells electronics and gadgets. As a customer service agent, I can only provide information related to our store. If you'd like to know more about AI, I'd be happy to help you find a resource that can provide more information on that topic.\n\nBut if you'd like to know more about the AI-related products we sell in our store, I'd be happy to help you with that!"